In [11]:
import math
import random
from enum import Enum

import networkx as nx

from mesa import Agent, Model
from mesa.time import StagedActivation
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
from mesa.space import NetworkGrid

from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from pyvis.network import Network

import math

from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter
from mesa.visualization.modules import ChartModule
from mesa.visualization.modules import NetworkModule
from mesa.visualization.modules import TextElement

import pandas as pd 
import numpy as np
import scipy.sparse as sps
import scipy as sp

import matplotlib.pyplot as plt

import multinetx as mx

%run Helper_functions.ipynb

In [12]:
# Function to create list of connected networks ('N>30')
def create_networks(networks, N=30):
    'Function to create the networks involved'
    # Return list with created networks
    network_list = []
    
    # Create IDs for agents
    shuffled = random.sample(range(0, N), N)
    shuffledf = random.sample(range(0, N), N)

    # Set roles business
    kingpin_main = shuffled[0]
    kingpin_list = shuffled[1:4]
    orphan_list = shuffled[4:8]
    broker_list = shuffled[8:16]
    dealer_list = shuffled[16:23]
    coordinator_list = shuffled[23:N]
    
    # set roles family/social
    father_main = shuffledf[0]
    cousin_list = shuffledf[1:4]
    brother_list = shuffledf[4:8]
    friend_list = shuffledf[8:16]
    driver_list = shuffledf[16:23]
    contact_list = shuffledf[23:N]
    
    # Create business/social role lists
    business_role_list = [kingpin_main, kingpin_list, orphan_list, broker_list, dealer_list, coordinator_list]
    family_role_list = [father_main, cousin_list, brother_list, friend_list, driver_list, contact_list]
    
    # Make dictionaries for attributes
    business_color_dict = make_dict(business_role_list, 'color')
    family_color_dict = make_dict(family_role_list, 'color')
    business_name_dict = make_dict(business_role_list, 'business_name')
    family_name_dict = make_dict(family_role_list, 'family_name')
    business_weight_dict = make_dict(business_role_list, 'weight')
    family_weight_dict = make_dict(family_role_list, 'weight')
        
    # Create each network
    for name in networks:
        
        # Start with empty network
        G = nx.empty_graph(N)
        
        # Set names, colours and weights
        nx.set_node_attributes(G, business_name_dict, "business_name")
        nx.set_node_attributes(G, family_name_dict, "family_name")
        nx.set_node_attributes(G, business_color_dict, "business_color")
        nx.set_node_attributes(G, family_color_dict, "family_color")
        nx.set_node_attributes(G, business_weight_dict, "business_weight")
        nx.set_node_attributes(G, family_weight_dict, "family_weight")
        
        # For business network
        if name == 'business':
            
            # Add edge lists including weight and colours
            add_edge_list(G, coordinator_list, G.nodes(), 0.1, 'pink', 30) 
            add_edge_list(G, dealer_list, G.nodes(), (1/6), 'green', 50) 
            add_edge_list(G, broker_list, G.nodes(), 0.25, 'blue', 70) 
            add_edge_list(G, orphan_list, G.nodes(), (1/3), 'yellow', 80, True) 
            add_edge_list(G, kingpin_list, G.nodes(), 0.25, 'orange', 100)
            add_edge_list(G, [kingpin_main], G.nodes(), 1, 'red', 100)

            # Append network for returning
            network_list.append(G)

            # Get node and edge colours for drawing
            business_values = [business_color_dict.get(node, 'k') for node in G.nodes()]
            edge_colors = nx.get_edge_attributes(G,'color').values()
            
            # Draw network
            nx.draw(G, with_labels=True, edge_color=edge_colors, node_color=business_values)
            plt.show()
            
        # For social network
        if name == 'social':
            
            # Add edge lists including weight and colours
            add_edge_list(G, driver_list, contact_list, 1, 'green', 50, True) 
            add_edge_list(G, friend_list, driver_list, 1, 'blue', 70, True) 
            add_edge_list(G, brother_list, friend_list, 1, 'yellow', 80, True) 
            add_edge_list(G, cousin_list, friend_list, 1, 'orange', 70, True) 
            add_edge_list(G, [father_main], G.nodes(), 1, 'red', 100, True) 

            # Append network for returning
            network_list.append(G)           
                
            # Get node and edge colours for drawing
            family_colors = nx.get_edge_attributes(G,'color').values()
            family_values = [family_color_dict.get(node, 'k') for node in G.nodes()]

            # Draw network
            nx.draw(G, with_labels=True, edge_color=family_colors, node_color=family_values)
            plt.show()

    return network_list